In [0]:
import tensorflow as tf


In [0]:
import requests
r = requests.get('https://api.ipify.org?format=json')
j = r.json()
print(j)


{'ip': '35.185.112.43'}


In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input,Conv2D,Dense,Flatten,Dropout
from tensorflow.keras.models import Model
import os

In [0]:
#Load in data

fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train,y_train), (x_test,y_test) = fashion_mnist.load_data()
x_train,x_test = x_train/255.0,x_test/255.0
print("x_train.shape:",x_train.shape)
print("x_test.shape:",x_test.shape)


x_train.shape: (60000, 28, 28)
x_test.shape: (10000, 28, 28)


In [0]:
#convolution expects h*w*c
x_train= np.expand_dims(x_train,-1)
x_test= np.expand_dims(x_test,-1)
print(x_train.shape)

(60000, 28, 28, 1)


In [0]:
#no of classes
k= len(set(y_train))
print("no of classes:",k)

no of classes: 10


In [0]:
#build the model

i= Input(shape= x_train[0].shape)
x= Conv2D(32, (3,3), strides=2, activation='relu')(i)
x= Conv2D(64, (3,3), strides=2, activation='relu')(x)
x= Conv2D(128, (3,3), strides=2, activation='relu')(x)
x= Flatten()(x)
x= Dropout(0.2)(x)
x= Dense(512,activation='relu')(x)
x= Dropout(0.2)(x)
x= Dense(k, activation='softmax')(x)

model= Model(i,x)

In [0]:
#Compile and Fit
#Need GPU
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
r= model.fit(x_train,y_train,validation_data=(x_test,y_test), epochs=15)


Epoch 1/15
1875/1875 [==============================] - 24s 13ms/step - loss: 0.5214 - accuracy: 0.8088 - val_loss: 0.4032 - val_accuracy: 0.8494
Epoch 2/15
1875/1875 [==============================] - 24s 13ms/step - loss: 0.3634 - accuracy: 0.8642 - val_loss: 0.3505 - val_accuracy: 0.8683
Epoch 3/15
1875/1875 [==============================] - 24s 13ms/step - loss: 0.3126 - accuracy: 0.8829 - val_loss: 0.3222 - val_accuracy: 0.8790
Epoch 4/15
1875/1875 [==============================] - 24s 13ms/step - loss: 0.2796 - accuracy: 0.8940 - val_loss: 0.2950 - val_accuracy: 0.8936
Epoch 5/15
1875/1875 [==============================] - 24s 13ms/step - loss: 0.2541 - accuracy: 0.9035 - val_loss: 0.2986 - val_accuracy: 0.8950
Epoch 6/15
1875/1875 [==============================] - 24s 13ms/step - loss: 0.2313 - accuracy: 0.9114 - val_loss: 0.3146 - val_accuracy: 0.8907
Epoch 7/15
1875/1875 [==============================] - 24s 13ms/step - loss: 0.2134 - accuracy: 0.9188 - val_loss: 0.2850 -

In [0]:
#Save the MOdel to temporary directory

import tempfile

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR,str(version))
print('export_path = {}\n'.format(export_path))
if os.path.isdir(export_path):
  print('\n Already saved a model, cleaning up\n')
  !rm -r {export_path}
tf.saved_model.save(model, export_path)

print('\nSaved model:')
!ls -l {export_path}

#protocol Buffer stands for  .pb extension

export_path = /tmp/1

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /tmp/1/assets

Saved model:
total 168
drwxr-xr-x 2 root root   4096 May 12 14:34 assets
-rw-r--r-- 1 root root 161950 May 12 14:34 saved_model.pb
drwxr-xr-x 2 root root   4096 May 12 14:34 variables


In [0]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_2'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28, 1)
        name: serving_default_input_2:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
W0512 14:40:00.414551 139836958431104 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/re

In [0]:
# echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow_model_server"
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg
# apt update

echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow_model_server tensorflow_model_server_universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \ curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add - as root

SyntaxError: ignored

In [0]:
!apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package tensorflow-model-server


In [0]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [0]:
%%bash --bg
nohup tensorflow_model_server \
--rest_api_port=8501\
--model name=fashion_model \
--model_base_path="{$MODEL_DIR}" >server.log 2>&1


Starting job # 0 in a separate thread.


In [0]:
!tail server.log

nohup: failed to run command 'tensorflow_model_server': No such file or directory


In [0]:
#labels mapping
labels= '''Tshirt/top
Trouser
Pullover
Dress
Coat
Sandal
Shirt
Sneaker
Bag
Ankle boot'''.split()